In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import h5py
from math import pi

from analysis.config import birds, h5_path_dict
from analysis.ExpData import ExpData

In [3]:
f = h5py.File(h5_path_dict['LMN73'][0].as_posix(), 'r')
exp_data = ExpData(f)